In [1]:
import onnx
from onnx import helper
from onnx import numpy_helper
from onnx import AttributeProto, TensorProto, GraphProto
import sys,getopt
import numpy as np 
import os
import matplotlib.pyplot as plt
import json


import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

from keras.models import * 
from keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.applications.vgg16 import preprocess_input,decode_predictions

Using TensorFlow backend.
/usr/local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarn

In [2]:
m = load_model('/data1/yolov3_tiny_416.h5')
m.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 416, 416, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 416, 416, 16) 432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 416, 416, 16) 64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 416, 416, 16) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

/usr/local/lib/python3.5/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [13]:
for layer in m.layers:
    print(layer.name)
    print(layer.input.name)
    print("====")

input_1
input_1:0
====
conv2d_1
input_1:0
====
batch_normalization_1
conv2d_1/convolution:0
====
leaky_re_lu_1
batch_normalization_1/cond/Merge:0
====
max_pooling2d_1
leaky_re_lu_1/LeakyRelu/Maximum:0
====
conv2d_2
max_pooling2d_1/MaxPool:0
====
batch_normalization_2
conv2d_2/convolution:0
====
leaky_re_lu_2
batch_normalization_2/cond/Merge:0
====
max_pooling2d_2
leaky_re_lu_2/LeakyRelu/Maximum:0
====
conv2d_3
max_pooling2d_2/MaxPool:0
====
batch_normalization_3
conv2d_3/convolution:0
====
leaky_re_lu_3
batch_normalization_3/cond/Merge:0
====
max_pooling2d_3
leaky_re_lu_3/LeakyRelu/Maximum:0
====
conv2d_4
max_pooling2d_3/MaxPool:0
====
batch_normalization_4
conv2d_4/convolution:0
====
leaky_re_lu_4
batch_normalization_4/cond/Merge:0
====
max_pooling2d_4
leaky_re_lu_4/LeakyRelu/Maximum:0
====
conv2d_5
max_pooling2d_4/MaxPool:0
====
batch_normalization_5
conv2d_5/convolution:0
====
leaky_re_lu_5
batch_normalization_5/cond/Merge:0
====
max_pooling2d_5
leaky_re_lu_5/LeakyRelu/Maximum:0
===

AttributeError: 'list' object has no attribute 'name'

In [ ]:
m = load_model('/data1/model_data/vgg16.h5')
model = onnx.load('/data1/model_data/vgg16.h5.onnx')
node_num = len(model.graph.node)

weights = {}
dim = {}
for i in range (node_num):
    
    


    # obtain node's name form model graph
    weight_name = model.graph.node[i].name
    
 
    # check whether current node contains weight(this include weight and bias)
    # if node's op_type is 'Constant', the node contains weight. 
    if model.graph.node[i].op_type == "Constant":

        # create a multiple dict step by step.
        weights[weight_name] = {}
        dim[weight_name] = {}            
        
        # obtain the weight 
        weights[weight_name] = model.graph.node[i].attribute[0].t.float_data
        # obtain the origin shape of current weight
        dim[weight_name] = model.graph.node[i].attribute[0].t.dims 
        # reshape the 1 dim weight to 4 dims weight if it should be. For those bias, just keep 1 dim.      
        weights[weight_name] = np.reshape(weights[weight_name], dim[weight_name])
        # reshape weights to keras'h .h file style. For those bias, do nothing.
        if weights[weight_name].ndim == 4:
            weights[weight_name] = weights[weight_name].transpose(2, 3, 1, 0)
            keras_weight_name = weight_name[:-7]
            keras_weights = m.get_layer(keras_weight_name).get_weights()[0]
        elif weights[weight_name].ndim == 2:
            keras_weight_name = weight_name[:-7]
            keras_weights = m.get_layer(keras_weight_name).get_weights()[0]
        else:
            keras_weight_name = weight_name[:-5]
            keras_weights = m.get_layer(keras_weight_name).get_weights()[1]
        
        if ((keras_weights == weights[weight_name]).all()):
            print('The onnx file weight value in layer' + ' ' + weight_name + ' is same as keras file')
        else:
            match = True
            print(keras_weights.shape)
            print(weight_name)
            for i_ in range(keras_weights.shape[0]):
                if i_ % 1000 == 999:
                    print("i_: {}".format(i_))
                onnx_layer_num = int(49 * i_ / keras_weights.shape[0]) + 49 * i_ % keras_weights.shape[0]
                if (keras_weights[i_] == weights[weight_name][onnx_layer_num]).all():
                    continue
                else:
                    print("not match !!! i_ : {}".format(i_))
                    match = False
                    break
            if match:
                print('The onnx file weight value in layer' + ' ' + weight_name + ' is same as keras file')
            